In [1]:
Parts Copyright (c) Chris Elroy extracted from
    https://github.com/JuliaSIMD/LoopVectorization.jl#usage

for fair and appropriate use here in a Jupyter notebook as per the MIT license.

Used the Julia release channel kernel current on 15th Apr 2024

The Julia is native compiled and the vector/matrix/tensors you might want are built in,
  so we do not need to roll any types by hand.
We can pull in some packages for optimizations.
This reflects the maturity of the Julia ecosystem.

UndefVarError: UndefVarError: `Parts` not defined

In [2]:
import Pkg;

Pkg.add("BenchmarkTools")
import BenchmarkTools

Pkg.add("LoopVectorization")
import LoopVectorization

Pkg.add("LinearAlgebra")
import LinearAlgebra


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


First we define the general test parameters.
Unlike the Mojo benchmark we do not create and free the matrices at every test.
We could, but neither version measures the time for memory management and
so there is no need to create and free.
C will be overwritten for each test.  A and B remain constant after random initialization.

In [3]:
using BenchmarkTools

M = 1024
N = 1024
K = 1024

C = zeros(Float32, M, N)
A = rand(Float32, M, K)
B = rand(Float32, K, N)

# it is conventional to count "+=" as 2 FLOPs even though the fused multiply-add is pipelined as one op.

FLOPcount = 2 * M * N * K

2147483648

The Naive, CmnDot, and vectorized versions are all together.

They are all short.  The vectorized (avx) version just has an @turbo macro added and takes away the @inbounds and @fastmath, it is otherwise the same as the CmnDot version.

In [4]:
using LoopVectorization

# adding a naive version similar to Mojo sample, to see what the optimizer does

function mygemm_naive!(C, A, B)
    @inbounds @fastmath for m ∈ axes(A,1), k ∈ axes(A,2)
        for n ∈ axes(B,2)
            C[m,n] += A[m,k] * B[k,n]
        end
    end
end

# citation to https://github.com/JuliaSIMD/LoopVectorization.jl#usage

function mygemm!(C, A, B)
    @inbounds @fastmath for m ∈ axes(A,1), n ∈ axes(B,2)
        Cmn = zero(eltype(C))
        for k ∈ axes(A,2)
            Cmn += A[m,k] * B[k,n]
        end
        C[m,n] = Cmn
    end
end

# just add turbo!

function mygemm_naiveavx!(C, A, B)
    @turbo for m ∈ axes(A,1), k ∈ axes(A,2)
        for n ∈ axes(B,2)
            C[m,n] += A[m,k] * B[k,n]
        end
    end
end

function mygemmavx!(C, A, B)
    @turbo for m ∈ axes(A,1), n ∈ axes(B,2)
        Cmn = zero(eltype(C))
        for k ∈ axes(A,2)
            Cmn += A[m,k] * B[k,n]
        end
        C[m,n] = Cmn
    end
end

mygemmavx! (generic function with 1 method)

The naive version is much slower than the CmnDot.

But when @turbo is used to vectorize them they become almost equally optimized
at around 81 GFLOPs, which is a more then 50x improvement (and 3x faster than the Mojo vectorized version)

In [5]:
elapsed = @belapsed mygemm_naive!(C, B, A)

println("Naive: ", elapsed, " secs, ", FLOPcount / 1e9 / elapsed, " GFLOPs")

elapsed = @belapsed mygemm!(C, B, A)

println("CmnDot: ", elapsed, " secs, ", FLOPcount / 1e9 / elapsed, " GFLOPs")

elapsed = @belapsed mygemm_naiveavx!(C, B, A)

println("Naive turbo: ", elapsed, " secs, ", FLOPcount / 1e9 / elapsed, " GFLOPs")

elapsed = @belapsed mygemmavx!(C, B, A)

print("CmnDot turbo: ", elapsed, " secs, ", FLOPcount / 1e9 / elapsed, " GFLOPs")

Naive: 6.993728659 secs, 0.30705847377085094 GFLOPs
CmnDot: 1.444738797 secs, 1.4864165428790654 GFLOPs
Naive turbo: 0.024527639 secs, 87.55362258878648 GFLOPs
CmnDot turbo: 0.023422773 secs, 91.6835785412769 GFLOPs

OK, rather than parallelizing by hand, which is possible with about the same coding effort as Mojo, lets just use Julia ecosystem where it is already done.

Import LinearAlgebra.  It comes complete with overloaded operators, so that as usual in Julia it reads just like your math textbooks would.

In [6]:
using LinearAlgebra

function mygemmBLAS!(C, A, B)
    C = A * B
end

mygemmBLAS! (generic function with 1 method)

Not playing any more.  Let's do this Julia style.

My CPU has 8 cores so I select 8 threads (16 hyperthreads runs worse, not better).

In [11]:
BLAS.set_num_threads(1)

elapsed = @belapsed mygemmBLAS!(C, B, A)

println("BLAS 1 thread : ", elapsed, " secs, ", FLOPcount / 1e9 / elapsed, " GFLOPs")

BLAS.set_num_threads(8)

elapsed = @belapsed mygemmBLAS!(C, B, A)

println("BLAS 8 threads: ", elapsed, " secs, ", FLOPcount / 1e9 / elapsed, " GFLOPs")

BLAS 1 thread : 0.016727589 secs, 128.37974725467012 GFLOPs
BLAS 8 threads: 0.002861231 secs, 750.5453589731134 GFLOPs


129 GFLOPs single thread is about 1.4x faster than the vectorized "mygemm" source.
750 GFLOPs 8 threaded is almost 5x faster, and 5x faster than the Mojo parallel version.

In [12]:
# sanity check that the calculation really happened..
# each value expected value is 0.25 * 1024 -> 256, with std dev 256/sqrt(1024) -> 8

C

1024×1024 Matrix{Float32}:
 256.283  262.353  254.391  253.787  …  260.341  259.083  250.08   260.396
 254.645  256.869  259.43   256.911     256.617  259.329  251.29   262.148
 250.619  245.617  243.974  238.565     244.804  246.429  241.587  253.329
 247.227  249.97   248.719  241.492     251.219  256.507  242.76   262.488
 254.726  264.42   263.715  258.729     261.764  265.687  253.729  265.699
 252.991  254.34   252.051  245.291  …  256.109  261.147  252.612  256.529
 261.584  253.605  255.758  252.27      262.825  260.035  256.663  263.216
 260.633  259.351  255.232  250.273     260.822  259.813  255.686  263.063
 260.304  255.054  255.753  254.197     265.911  259.335  249.991  263.131
 252.687  255.345  251.395  254.461     257.248  254.255  245.558  258.976
   ⋮                                 ⋱    ⋮                        
 264.168  260.992  260.542  263.356  …  265.777  266.97   259.857  273.076
 258.738  258.47   260.913  253.198     262.282  263.603  257.795  269.07
 258.2